##  Optimization manual steps

In [1]:
import random
import numpy as np
import pprint

from mycrograd_debug.engine_debug import Value
from mycrograd_debug.nn_debug import MLP
from mycrograd_debug.drawviz_debug import (
    draw_dot,
    draw_nn,
    print_all_values,
    print_my_params,
)

np.random.seed(1337)
random.seed(1337)
number_of_iterations = 10
pp = pprint.PrettyPrinter(indent=4)
global activation
global loss
makeimg = False
debug_parameters = True
debug_values = False
debug_steps=True

#### Multi Layer Perceptron 1-2-1

In [2]:
# initialize a model
nin = 1  # number of inputs
nout = 1  # number of outputs
Value.value_counter = 0

model = MLP(nin, [2, nout], weightsinit=2, lastReLU=False, debug_bw=False)
print("parameters")
print_my_params(model)
xinumbers = list(range(4, 4 + nin))
xinput = [Value(x, type="i") for x in xinumbers]
print("inputs= ", xinput)
xtarget = Value(1.2, type="t")  # desired targets
print("targets= ", xtarget)

Module nn MLP: structure [1, 2, 1]
parameters
 name lay neu ty   data   grad
 v001  L1  N1 w1   0.50   0.00
 v002  L1  N1  b   0.00   0.00
 v003  L1  N2 w1   0.50   0.00
 v004  L1  N2  b   0.00   0.00
 v005  L2  N1 w1   0.60   0.00
 v006  L2  N1 w2   1.20   0.00
 v007  L2  N1  b   0.00   0.00
inputs=  [Value(name=v008,layernumber=,neuronnumber=,weightnumber=,type=i,data=4, grad=0)]
targets=  Value(name=v009,layernumber=,neuronnumber=,weightnumber=,type=t,data=1.2, grad=0)


In [3]:
# loss function single MLP
def loss_single(activation, target):
    total_loss = (activation - target)*(activation - target)
    total_loss.type="l"
    return total_loss

In [4]:
def makeImage(filename):
    # print(filename)
    dot1=draw_dot(activation)
    dot1.render("images/"+filename , format="svg", view=True)
    dot2=draw_nn(xinput, model)
    dot2.render("images/nn/"+filename, format="svg", view=True)

In [5]:
def act():
    #### forward pass
    global activation
    global loss
    activation = model(xinput)
    # total_loss = activation
    loss = loss_single(activation, xtarget)

    if debug_parameters:
        print_my_params(model)
    if debug_values:
        print_all_values(activation)


def zeroGrad():
    global activation
    model.zero_grad()
    for i in xinput:
        i.grad = 0
    if debug_parameters:
        print_my_params(model)
    if debug_values:
        print_all_values(activation)


def back():
    #### backward pass
    global activation
    activation.backward()
    if debug_parameters:
        print_my_params(model)
    if debug_values:
        print_all_values(activation)


def upd():
    #### update
    global activation
    for p in model.parameters():
        p.data += -0.05 * p.grad
    if debug_parameters:
        print_my_params(model)
    if debug_values:
        print_all_values(activation)


def opt():
    global loss

    for k in range(number_of_iterations):
        print("start step %d" % k)

        if debug_steps:
            print("step %d loss calc" % k)
        act()
        if makeimg:
            makeImage("opt_01_step%d_1loss" % k)

        if debug_steps:
            print("step %d zero grad" % k)
        zeroGrad()
        if makeimg:
            makeImage("opt_01_step%d_2zero" % k)

        if debug_steps:
            print("step %d backward" % k)
        back()
        if makeimg:
            makeImage("opt_01_step%d_3back" % k)

        if debug_steps:
            print("step %d update" % k)
        upd()
        if makeimg:
            makeImage("opt_01_step%d_3upda" % k)

        print(f"step %3d output %6.4f loss %6.4f" % (k, activation.data, loss.data))

In [6]:
opt()

start step 0
step 0 loss calc
 name lay neu ty   data   grad
 v001  L1  N1 w1   0.50   0.00
 v002  L1  N1  b   0.00   0.00
 v003  L1  N2 w1   0.50   0.00
 v004  L1  N2  b   0.00   0.00
 v005  L2  N1 w1   0.60   0.00
 v006  L2  N1 w2   1.20   0.00
 v007  L2  N1  b   0.00   0.00
step 0 zero grad
 name lay neu ty   data   grad
 v001  L1  N1 w1   0.50   0.00
 v002  L1  N1  b   0.00   0.00
 v003  L1  N2 w1   0.50   0.00
 v004  L1  N2  b   0.00   0.00
 v005  L2  N1 w1   0.60   0.00
 v006  L2  N1 w2   1.20   0.00
 v007  L2  N1  b   0.00   0.00
step 0 backward
 name lay neu ty   data   grad
 v001  L1  N1 w1   0.50   2.40
 v002  L1  N1  b   0.00   0.60
 v003  L1  N2 w1   0.50   4.80
 v004  L1  N2  b   0.00   1.20
 v005  L2  N1 w1   0.60   2.00
 v006  L2  N1 w2   1.20   2.00
 v007  L2  N1  b   0.00   1.00
step 0 update
 name lay neu ty   data   grad
 v001  L1  N1 w1   0.38   2.40
 v002  L1  N1  b  -0.03   0.60
 v003  L1  N2 w1   0.26   4.80
 v004  L1  N2  b  -0.06   1.20
 v005  L2  N1 w1   0.50 